<a href="https://colab.research.google.com/github/nabihsabeh85/Final-Project-/blob/main/INSTRUCTOR%E2%80%99S_COPILOT_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 nltk gensim gradio openai==0.28.0 googlesearch-python requests

import PyPDF2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import re
import gradio as gr
import openai
from googlesearch import search
import requests

nltk.download('punkt')
nltk.download('stopwords')

def preprocess(text):
    # Lowercasing
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    return tokens

# Securely input your OpenAI API key
api_key = "enter your API Key "
openai.api_key = api_key

def query_gpt4(topic):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant that give an AI teacher a new ideas for teaching."},
            {"role": "user", "content": f"Based on the following topic tokens: {topic}, provide new real-life ideas to help the teacher make explanations better. Break down the response by subject, and for each subject, include a use case, a real-life example, and discussion ideas. Format the response using the following structure:\n\n1. [Subject Name]\n\n##Use case\n[Use case description]\n\n##Real-life example\n[Real-life example description]\n\n##Discussion ideas\n[Discussion ideas description]\n\n##Related links:\n[Trending related link]\n\nRepeat this structure for each subject."}
        ],
        max_tokens=1000
    )
    return response['choices'][0]['message']['content']

def get_trending_link(subject):
    query = f"{subject} machine learning"
    for link in search(query, num_results=10):  # Increase the number of results to search for a working link
        try:
            response = requests.get(link)
            if response.status_code == 200:
                return link
        except:
            continue
    return None

def generate_report(pdf_file):
    pdf_reader = PyPDF2.PdfReader(pdf_file.name)
    num_pages = len(pdf_reader.pages)
    full_text = ""
    for page in range(num_pages):
        full_text += pdf_reader.pages[page].extract_text()

    tokens = preprocess(full_text)
    explanation = query_gpt4(tokens)

    subjects = re.split(r'(?=\d+\.)', explanation)[1:]  # Split the explanation into subjects
    final_report = ""
    for subject in subjects:
        subject_lines = subject.strip().split('\n')
        subject_name = subject_lines[0].strip()
        subject_content = '\n'.join(subject_lines[1:])
        trending_link = get_trending_link(subject_name)
        if trending_link:
            final_report += f"{subject_name}\n\n{subject_content.replace('##Trending related subjects:', '##Related links:')}\n[Click here for related link]({trending_link})\n\n"
        else:
            final_report += f"{subject_name}\n\n{subject_content.replace('##Trending related subjects:', '##Related links:')}\n\n"

    return final_report

with gr.Blocks(title="AIEd Enhancer") as iface:
    gr.Markdown("Upload a PDF and get new ideas for teaching based on the extracted topics.")

    with gr.Row():
        pdf_file = gr.File(label="Upload PDF")

    with gr.Row():
        explanation_textbox = gr.Markdown(label="Report")

    pdf_file.change(generate_report, inputs=pdf_file, outputs=explanation_textbox)

iface.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0299c3858c00f35d9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
